In [ ]:
import random
import unittest

# Define constants for categories and their corresponding scores
CATEGORIES = [
    "Ones", "Twos", "Threes", "Fours", "Fives", "Sixes",
    "Three of a Kind", "Four of a Kind", "Full House",
    "Small Straight", "Large Straight", "Yahtzee", "Chance"
]

# Roll Dice function
def roll_dice(num_dice):
    """
    Simulates rolling a given number of dice.

    Args:
    - num_dice (int): The number of dice to roll.

    Returns:
    - list: A list containing the results of the dice roll.
    """
    return [random.randint(1, 6) for _ in range(num_dice)]

# Calculate Score function
def calculate_score(dice, category):
    """
    Calculates the score for a given roll according to the selected category.

    Args:
    - dice (list): A list containing the results of the dice roll.
    - category (str): The category to score the roll in.

    Returns:
    - int: The calculated score for the given roll and category.
    """
    def count_occurrences():
        counts = [0] * 6
        for die in dice:
            counts[die - 1] += 1
        return counts

    if category in ["Ones", "Twos", "Threes", "Fours", "Fives", "Sixes"]:
        return dice.count(int(category[-1])) * int(category[-1])
    elif category == "Three of a Kind":
        if max(count_occurrences()) >= 3:
            return sum(dice)
        else:
            return 0
    elif category == "Four of a Kind":
        if max(count_occurrences()) >= 4:
            return sum(dice)
        else:
            return 0
    elif category == "Full House":
        counts = count_occurrences()
        if 2 in counts and 3 in counts:
            return 25
        else:
            return 0
    elif category == "Small Straight":
        if [1, 1, 1, 1] in [sorted(dice[i:i+4]) for i in range(len(dice) - 3)]:
            return 30
        else:
            return 0
    elif category == "Large Straight":
        if sorted(dice) in [[1, 2, 3, 4, 5], [2, 3, 4, 5, 6]]:
            return 40
        else:
            return 0
    elif category == "Yahtzee":
        if max(count_occurrences()) == 5:
            return 50
        else:
            return 0
    elif category == "Chance":
        return sum(dice)
    else:
        return 0  # Invalid category

# Keep Dice function
def keep_dice():
    """
    Allows the player to select which dice to keep after a roll.
    """
    while True:
        try:
            kept_dice = list(map(int, input("Enter indices of dice to keep (e.g., '1 3 4'), or 'r' to roll again: ").split()))
            if all(1 <= die <= 5 for die in kept_dice):
                return kept_dice
            else:
                print("Invalid indices. Please enter indices between 1 and 5.")
        except ValueError:
            if input("Invalid input. Do you want to roll again? (y/n): ").lower() != 'y':
                break

# Display Scorecard function
def display_scorecard(scorecard):
    """
    Displays the current state of the scorecard.

    Args:
    - scorecard (dict): A dictionary representing the current state of the scorecard.
    """
    print("Scorecard:")
    for category, score in scorecard.items():
        print(f"{category}: {score if score is not None else 'Not filled'}")

# Choose Category function
def choose_category(scorecard):
    """
    Prompts the player to choose a category to score their roll.

    Args:
    - scorecard (dict): A dictionary representing the current state of the scorecard.

    Returns:
    - str: The chosen category.
    """
    available_categories = [category for category, score in scorecard.items() if score is None]
    while True:
        print("Available categories:", available_categories)
        choice = input("Choose a category to score your roll: ").strip()
        if choice in available_categories:
            return choice
        else:
            print("Invalid category. Please choose from the available categories.")

# Check End Game function
def check_end_game(scorecard):
    """
    Checks if the game has ended by filling out the entire scorecard.

    Args:
    - scorecard (dict): A dictionary representing the current state of the scorecard.

    Returns:
    - bool: True if the game has ended, False otherwise.
    """
    return all(score is not None for score in scorecard.values())

# Bot Strategy function
def bot_strategy(scorecard, dice):
    """
    Implements the bot's strategy for choosing which category to score in.

    Args:
    - scorecard (dict): A dictionary representing the current state of the scorecard.
    - dice (list): A list containing the results of the dice roll.

    Returns:
    - str: The chosen category.
    """
    # For simplicity, the bot will choose the category with the highest potential score
    available_categories = [category for category, score in scorecard.items() if score is None]
    max_score = -1
    chosen_category = None
    for category in available_categories:
        score = calculate_score(dice, category)
        if score > max_score:
            max_score = score
            chosen_category = category
    return chosen_category

# Main Game Loop function
def main():
    """
    Controls the flow of the game.
    """
    scorecard = {category: None for category in CATEGORIES}

    while not check_end_game(scorecard):
        print("\nPlayer's turn:")
        dice = roll_dice(5)
        print("Rolling dice:", dice)
        keep_dice()
        category = choose_category(scorecard)
        score = calculate_score(dice, category)
        scorecard[category] = score
        display_scorecard(scorecard)

        print("\nBot's turn:")
        dice = roll_dice(5)
        print("Rolling dice:", dice)
        category = bot_strategy(scorecard, dice)
        score = calculate_score(dice, category)
        scorecard[category] = score
        display_scorecard(scorecard)

    print("Game Over")

# Unit tests
class TestYahtzee(unittest.TestCase):
    def test_calculate_score(self):
        # Test ones, twos, ..., sixes
        self.assertEqual(calculate_score([1, 1, 1, 1, 1], "Ones"), 5)
        self.assertEqual(calculate_score([2, 2, 2, 2, 2], "Twos"), 10)
        self.assertEqual(calculate_score([3, 3, 3, 3, 3], "Threes"), 15)
        self.assertEqual(calculate_score([4, 4, 4, 4, 4], "Fours"), 20)
        self.assertEqual(calculate_score([5, 5, 5, 5, 5], "Fives"), 25)
        self.assertEqual(calculate_score([6, 6, 6, 6, 6], "Sixes"), 30)

        # Test three of a kind, four of a kind, full house, small straight, large straight, yahtzee, chance
        self.assertEqual(calculate_score([1, 1, 1, 2, 2], "Three of a Kind"), 7)
        self.assertEqual(calculate_score([1, 1, 1, 2, 2], "Four of a Kind"), 8)
        self.assertEqual(calculate_score([1, 1, 2, 2, 2], "Full House"), 25)
        self.assertEqual(calculate_score([1, 2, 3, 4, 5], "Small Straight"), 30)
        self.assertEqual(calculate_score([2, 3, 4, 5, 6], "Large Straight"), 40)
        self.assertEqual(calculate_score([1, 1, 1, 1, 1], "Yahtzee"), 50)
        self.assertEqual(calculate_score([1, 2, 3, 4, 5], "Chance"), 15)

        # Test invalid category
        self.assertEqual(calculate_score([1, 2, 3, 4, 5], "Invalid"), 0)

if __name__ == "__main__":
    unittest.main()
